# `anato-mesh` in Jupyter Notebook 

`anato_mesh.py` contains the main functions for calculating the partition-level curvatures using python to reproduce the algorithm originally published by K. Khabaz here: https://journals.plos.org/ploscompbiol/article?id=10.1371/journal.pcbi.1011815

In [1]:
from anato_mesh import *
from anato_viz import *
import plotly.express as px
import warnings
#from pandas.errors import SettingWithCopyWarning
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter(action='ignore', category=UserWarning)
warnings.simplefilter(action='ignore', category=RuntimeWarning)
#warnings.simplefilter(action='ignore', category=SettingWithCopyWarning)

In [2]:
#- Pocivavsek lab parent directory
parent_path = 'Z:\\aorta\\thoracic'
#- Cohort filter
group_str = ['GE']
#- Mesh filter
file_str = ['M10']

<table>
    <tr>
        <th style="text-align:center">quantities</th>
        <th style="text-align:center">Equation</th>
    </tr>
    <tr>
        <td style="text-align:center">'Gaussian'</td>
        <td style="text-align:center">$$\displaystyle k_1 \cdot k_2$$</td>
    </tr>
    <tr>
        <td style="text-align:center">'Mean'</td>
        <td style="text-align:center">$$\displaystyle 0.5 \cdot (k_1 + k_2)$$</td>
    </tr>
    <tr>
        <td style="text-align:center">'IntGaussian'</td>
        <td style="text-align:center">$$\displaystyle (k_1 \cdot k_2) \cdot A$$</td>
    </tr>
    <tr>
        <td style="text-align:center">'IntMeanSquared'</td>
        <td style="text-align:center">$$\displaystyle \left(0.5 \cdot (k_1 + k_2)\right)^2 \cdot A$$</td>
    </tr>
    <tr>
        <td style="text-align:center">'Willmore'</td>
        <td style="text-align:center">$$\displaystyle 4 \cdot \left(0.5 \cdot (k_1 + k_2)\right)^2 \cdot A - 2 \cdot (k_1 \cdot k_2) \cdot A$$</td>
    </tr>
    <tr>
        <td style="text-align:center">'Casorati'</td>
        <td style="text-align:center">$$\displaystyle \sqrt{0.5 \cdot (k_1^2 + k_2^2)}$$</td>
    </tr>
    <tr>
        <td style="text-align:center">'ShapeIndex'</td>
        <td style="text-align:center">$$\displaystyle \frac{2}{\pi} \cdot \arctan\left(\frac{k_2 + k_1}{k_2 - k_1}\right)$$</td>
    </tr>
</table>


You must specify at least one variable in `quantitites` and must also define the `point_removal` method. When in doubt, use the `curvature` method because it is applicable to all geometries. The `thoracic` specific point removal process is still buggy, overly empirical, and is being fixed. 

In [3]:
quantities = ['Gaussian', 'Mean', 'IntGaussian', 'IntMeanSquared', 'Willmore', 'Casorati', 'ShapeIndex']
point_removal = 'curvature'

The number of surface partitions for each surface is determined by the following equation. `m_set` provides an easily adjustable parameter for further partition scaling and allows you to calculate more than one scaling at once. The optimized value for the thoracic aorta TEVAR dataset is `m=1`.

$$partitions = m \times \left(\frac{SA}{R^2}\right)$$

In [4]:
m_set = [5]

#### Executing the batch run: 

In [5]:
results, scan_dict = GetAnatoMeshResults(parent_path, group_str, file_str, point_removal, quantities, m_set)

Organizing paths and file names:
Adding KK1_1_M10.parquet
Adding KK12_7_M10.parquet
Adding KK16_6_M10.parquet
Adding KK23_5_M10.parquet
Adding KK32_0_M10.parquet
Adding KY4_1_M10.parquet
Adding KY7_1_M10.parquet
Adding SA2_1_M10.parquet
Starting GetAortaMeshResults: the top most progress bar is for all calculations and the progress bars below are for parallel processes.


Finished.


In [6]:
scan_dict

{5: {'KK1_1_M10':              X           Y           Z  Gaussian      Mean  IntGaussian  \
  0    -6.442200  155.093293  184.662005 -0.000026  0.017236    -0.002680   
  1    62.721240  170.646527   72.968562  0.000061  0.018899     0.004933   
  2    -0.071497  160.047360   31.627558 -0.000333  0.018334    -0.029324   
  3    25.432397  209.187738  139.239527  0.000016  0.012443     0.001668   
  4   -41.184561  107.595722  150.373040  0.000622  0.029100     0.080596   
  ..         ...         ...         ...       ...       ...          ...   
  545  -8.198028  127.932783   -3.361545  0.000235  0.018351     0.021917   
  546  50.842352  210.721902  172.958075  0.000280  0.022316     0.021629   
  547  71.840505  188.440181   92.012413 -0.000501  0.023497    -0.053618   
  548  18.533862  162.345417   60.546001 -0.000806  0.017948    -0.095152   
  549  26.559067  126.451317  175.084291  0.000045  0.027834     0.003513   
  
       IntMeanSquared  Willmore  Casorati  ShapeIndex  Nu

### If you just want data, stop here. 
#### Everything after this are postprocessing steps including normalization, meta-data integration, visualization, and exporting.

Normalizing the results. This works by first splitting up the compound name using the `SplitUpScanName` function and then normalizing the variables in the 2nd and 3rd function inputs to the `GroupsNormalize` function by the group of datapoints that contain the string sequence in the 4th function input. This is normally `KY` patients for TEVAR and `JX` patients for EVAR in our data. 

In [ ]:
results_norm = GroupsNormalize(SplitUpScanName(results), 'Casorati_Mean', 'IntGaussian_Fluct', 'JX')

Merging meta data files into the anato-mesh results.

In [ ]:
#- Meta data integration 
directory = 'Z:\\aorta\\meta_data'
file_name = 'MasterMetaData.xlsx'

#- Cohort meta data group names
cohort_list = ['UC_NORMAL', 'UC_PEDS', 'UC_TEVAR', 'ENDOSPAN', 'MEDTRONIC', 'GORE_801', 'GORE_802', 'GORE_803']

#- Integer columns to be converted to strings for discrete visualization
cat_columns = ['Label', 'Outcome']

In [ ]:
results_meta = MergeMetaData(directory, file_name, cohort_list, results_norm, cat_columns)

Ploting and saving the data. 

In [ ]:
fig = px.scatter(results_norm, x='Mean_Radius', y='Total_Fluct_Norm')#, color='Label', hover_data=['Scan_Name'])
fig.show()

In [ ]:
SaveResultsToXLSX('path', 'data.xlsx', results_meta)

In [ ]:
SaveScanDictToXLSX('path', 'scan_dict.xlsx', scan_dict) 